In [34]:
import pandas as pd  # pour manipuler les matrices de distance
import numpy as np

# 1. Introduction

In [35]:
#2 : 
# on czmmence par les matrices M1 et M2 du fichier donné
M1 = pd.DataFrame([[0, 8, 7, 12], [8, 0, 9, 14], [7, 9, 0, 11], [12, 14, 11, 0]],
                  columns=['A', 'B', 'C', 'D'], index=['A', 'B', 'C', 'D'])

M2 = pd.DataFrame([[0, 2, 3, 8, 14, 18], [2, 0, 3, 8, 14, 18], [3, 3, 0, 8, 14, 18],
                   [8, 8, 8, 0, 14, 18], [14, 14, 14, 14, 0, 18], [18, 18, 18, 18, 18, 0]],
                  columns=['A', 'B', 'C', 'D', 'E', 'F'], index=['A', 'B', 'C', 'D', 'E', 'F'])

#fonction pour la vérification de si matrice additive
def est_additive(matrice_distance):  
    n = matrice_distance.shape[0]  # pour le nombre d'espèces distinctes
    for i in range(n):
        for j in range(i + 1, n):
            for k in range(j + 1, n):
                for l in range(k + 1, n):
                    dist_ij_kl = [
                        matrice_distance.iloc[i, j] + matrice_distance.iloc[k, l],
                        matrice_distance.iloc[i, k] + matrice_distance.iloc[j, l],
                        matrice_distance.iloc[i, l] + matrice_distance.iloc[j, k]
                    ]
                    if max(dist_ij_kl) != sum(sorted(dist_ij_kl)[:2]):  #condition de quatre points
                        return False
    return True

#fonction pour la vérification de si matrice ultramétrique
def est_ultramétrique(matrice_distance):
    n = matrice_distance.shape[0] 
    for i in range(n):
        for j in range(i + 1, n):
            for k in range(j + 1, n):
                d_ij = matrice_distance.iloc[i, j]
                d_ik = matrice_distance.iloc[i, k]
                d_jk = matrice_distance.iloc[j, k]
                if not (d_ij <= max(d_ik, d_jk) and d_ik <= max(d_ij, d_jk) and d_jk <= max(d_ij, d_ik)):
                    return False
    return True

# fonction pour le calcul de la somme des distances pour une espèce et pour toutes les espèces
def somme_distances_espece(matrice_distance, espece): 
    indice = matrice_distance.index.get_loc(espece)  #on identifie l'indice de l'espece
    somme_distances = matrice_distance.iloc[indice].sum()  #somme pour l'espece
    return somme_distances

def somme_distances_toutes_especes(matrice_distance):
    sommes = {}
    for espece in matrice_distance.index:
        sommes[espece] = somme_distances_espece(matrice_distance, espece)
    return sommes


In [36]:
#tests des fonctions 

# test si M1 est additive /ultramétrique
print("M1 est additive :", est_additive(M1))
print("M1 est ultramétrique :", est_ultramétrique(M1))

#test si M2 est additive puis ultramétrique
print("M2 est additive :", est_additive(M2))
print("M2 est ultramétrique :", est_ultramétrique(M2))

# test sommes de distances pour chaque espece dans M1 puis dans M2
print("somme des distances pour chaque espèce dans M1 :", somme_distances_toutes_especes(M1))
print("somme des distances pour chaque espèce dans M2 :", somme_distances_toutes_especes(M2))

M1 est additive : False
M1 est ultramétrique : False
M2 est additive : False
M2 est ultramétrique : True
somme des distances pour chaque espèce dans M1 : {'A': 27, 'B': 31, 'C': 27, 'D': 37}
somme des distances pour chaque espèce dans M2 : {'A': 45, 'B': 45, 'C': 46, 'D': 56, 'E': 74, 'F': 90}


# 2. UPGMA

In [37]:
#2 : 

#matrice M3 avec noms d'espèces
M3 = pd.DataFrame([[0, 19, 27, 8, 33, 18, 13], 
                   [19, 0, 31, 18, 36, 1, 13],
                   [27, 31, 0, 26, 41, 32, 29],
                   [8, 18, 26, 0, 31, 17, 14],
                   [33, 36, 41, 31, 0, 35, 28],
                   [18, 1, 32, 17, 35, 0, 12],
                   [13, 13, 29, 14, 28, 12, 0]], 
                  columns=['A', 'B', 'C', 'D', 'E', 'F', 'G'],
                  index=['A', 'B', 'C', 'D', 'E', 'F', 'G'])

#fonction pour trouver les deux clusters les plus proches
def trouver_clusters_proches(matrice_distance):
    min_distance = np.inf
    clusters_proches = None
    for i in range(len(matrice_distance)):
        for j in range(i + 1, len(matrice_distance)):
            if matrice_distance.iloc[i, j] < min_distance:
                min_distance = matrice_distance.iloc[i, j]
                clusters_proches = (matrice_distance.index[i], matrice_distance.index[j])
    return clusters_proches, min_distance

# Fonction pour mettre à jour la matrice de distance après fusion de deux clusters
def mettre_a_jour_matrice(matrice_distance, cluster1, cluster2, nouveau_nom):
    nouvelles_distances = []
    for espece in matrice_distance.index:
        if espece not in [cluster1, cluster2]:
            distance_moyenne = (matrice_distance[cluster1][espece] + matrice_distance[cluster2][espece]) / 2
            nouvelles_distances.append(distance_moyenne)
    matrice_distance = matrice_distance.drop([cluster1, cluster2], axis=0).drop([cluster1, cluster2], axis=1)
    matrice_distance[nouveau_nom] = nouvelles_distances
    nouvelle_colonne = pd.Series(nouvelles_distances + [0], index=matrice_distance.columns)
    matrice_distance.loc[nouveau_nom] = nouvelle_colonne
    return matrice_distance

# Fonction UPGMA pour créer un arbre en format Newick
def UPGMA(matrice_distance):
    matrice = matrice_distance.copy()
    arbres = {nom: nom for nom in matrice.index}
    compteur = 1
    while len(matrice) > 1:
        (cluster1, cluster2), distance = trouver_clusters_proches(matrice)
        nouveau_nom = f"Cluster{compteur}"
        arbres[nouveau_nom] = f"({arbres.pop(cluster1)}:{distance/2},{arbres.pop(cluster2)}:{distance/2})"
        matrice = mettre_a_jour_matrice(matrice, cluster1, cluster2, nouveau_nom)
        compteur += 1
    return list(arbres.values())[0] + ";"

# Test de la fonction UPGMA avec la matrice M3
print("Arbre en format Newick :", UPGMA(M3))


Arbre en format Newick : (E:18.21875,(C:14.1875,((A:4.0,D:4.0):7.875,(G:6.25,(B:0.5,F:0.5):6.25):7.875):14.1875):18.21875);


# 3. NJ

In [38]:
M4 = pd.DataFrame([[0, 2, 4, 6, 6, 8], 
                   [2, 0, 4, 6, 6, 8], 
                   [4, 4, 0, 6, 6, 8], 
                   [6, 6, 6, 0, 4, 8], 
                   [6, 6, 6, 4, 0, 8], 
                   [8, 8, 8, 8, 8, 0]], 
                  columns=['A', 'B', 'C', 'D', 'E', 'F'], 
                  index=['A', 'B', 'C', 'D', 'E', 'F'])

In [39]:
def calculer_u_i(matrice_distance, espece):
    n = len(matrice_distance)
    return matrice_distance.loc[espece].sum() / (n - 2)  #(on divise selon l'aglo NJ)

#test avec m4
print("u_i pour A:", calculer_u_i(M4, 'A'))


u_i pour A: 6.5


In [40]:

#on calcul Qij pour deux clusters donnés
def calculer_Qij(matrice_distance, cluster1, cluster2):
    n = len(matrice_distance)
    ui = calculer_ui(matrice_distance, cluster1)
    uj = calculer_ui(matrice_distance, cluster2)
    return (n - 2) * matrice_distance.loc[cluster1, cluster2] - ui - uj

#test
print("Qij pour A et B:", calculer_Qij(M4, 'A', 'B'))


Qij pour A et B: -5.0


In [41]:

#fonction de NJ pour avoir un arbre en format newick
def NJ(matrice_distance):
    matrice = matrice_distance.copy()
    arbres = {nom: nom for nom in matrice.index}
    compteur = 1

    while len(matrice) > 2:         #(pour trouver les clusters les plus proches)
        min_Qij = np.inf
        clusters_proches = None
        for i in range(len(matrice)):
            for j in range(i + 1, len(matrice)):
                cluster1, cluster2 = matrice.index[i], matrice.index[j]
                Qij = calculer_Qij(matrice, cluster1, cluster2)
                if Qij < min_Qij:
                    min_Qij = Qij
                    clusters_proches = (cluster1, cluster2)

        cluster1, cluster2 = clusters_proches
        distance = matrice.loc[cluster1, cluster2]
        u1 = calculer_ui(matrice, cluster1)
        u2 = calculer_ui(matrice, cluster2)

        #longueurs des branches en direction du nouveau nœud.
        branche1 = (distance / 2) + ((u1 - u2) / 2)
        branche2 = distance - branche1

        #mise en place du nouveau bloc en format Newick
        nouveau_nom = f"Cluster{compteur}"
        arbres[nouveau_nom] = f"({arbres.pop(cluster1)}:{branche1},{arbres.pop(cluster2)}:{branche2})"
        
        # mise à jour de la matrice
        nouvelles_distances = []
        for espece in matrice.index:
            if espece not in [cluster1, cluster2]:
                distance_moyenne = (matrice[cluster1][espece] + matrice[cluster2][espece]) / 2
                nouvelles_distances.append(distance_moyenne)
        matrice = matrice.drop([cluster1, cluster2], axis=0).drop([cluster1, cluster2], axis=1)
        matrice[nouveau_nom] = nouvelles_distances
        nouvelle_colonne = pd.Series(nouvelles_distances + [0], index=matrice.columns)
        matrice.loc[nouveau_nom] = nouvelle_colonne

        compteur += 1

    #pour inclure la branche finale entre les deux groupes restés.
    cluster1, cluster2 = matrice.index[0], matrice.index[1]
    distance = matrice.loc[cluster1, cluster2] / 2
    arbre_newick = f"({arbres.pop(cluster1)}:{distance},{arbres.pop(cluster2)}:{distance});"

    return arbre_newick

#test NJ avec M4
print("Arbre en format Newick :", NJ(M4))


Arbre en format Newick : ((D:2.0,E:2.0):3.5,(F:5.0,(C:2.0,(A:1.0,B:1.0):2.0):3.0):3.5);
